In [ ]:
from __future__ import annotations

In [ ]:
import abc
import weakref
from dataclasses import dataclass
from typing import Any, ClassVar, Generic, Optional, Union, get_args, get_origin

In [ ]:
import pandas as pd

In [ ]:
from eos.data_io.eos_struct import ItemT, PoolQuery

In [ ]:
@dataclass
class Pool(abc.ABC, Generic[ItemT]):
    """
    Pool is the abstract storage for pooling the real-time data from the cloud.
    Pool provides the following features:
    - load(): initialize the pool interface
    - close(): destructor
    - store(): Store the data in the pool
    - delete(id): remove the data with id in the pool
    - find(id): Get the data from the pool
    - sample(size, query: Optional[dict]=None): Sample a batch of data from the pool
    - count(query: Optional[dict] = None): Count the number of data in the pool

    - index: pointer to the current position in the pool
            python int has 24 bytes which is sufficient for ObjectID in MongoDB

    """

    _type_T: ClassVar[str] = ""
    _cnt: int = 0  # number of records in the pool. Calling count() is expensive and it will update this.

    def __init_subclass__(cls):
        # get the concrete type in derived class
        cls._type_T = get_args(cls.__orig_bases__[0])[0].__name__  # type: ignore
        # print(get_origin(cls.__orig_bases__[0]).__name__)
        # print(cls.__bases__[0].__name__)  # type: ignore

        # print(
        #     f"Pool.__init_subclass__(): {cls._type_T} in {cls.__name__} from {cls.__base__.__name__}"
        # )

    def __post_init__(self):
        """User weakref finalizer to make sure close is called when the object is destroyed"""
        self._finalizer = weakref.finalize(self, self.close)

    @abc.abstractmethod
    def load(self):
        """Initialize the pool interface
        - connect to db
        - init
        """

    @abc.abstractmethod
    def close(self):
        """close the pool, for destructor"""

    @abc.abstractmethod
    def store(self, item: ItemT):
        """Deposit an item (record) into the pool"""

    @abc.abstractmethod
    def delete(self, idx):
        """
        delete an itme by id or name.
        """

    @abc.abstractmethod
    def _count(self, query: Optional[PoolQuery] = None) -> int:
        """
        Count the number of records in the db.
        rule is an optional dictionary specifying a rule or
        a pipeline in mongodb
        query = {
            vehicle_id: str = "VB7",
            driver_id: str = "longfei-zheng",
            dt_start: datetime = None,
            dt_end: datetime = None,
            }
        """

    @property
    def cnt(self) -> int:
        """Number of records in the pool"""
        return self._cnt

    @cnt.setter
    def cnt(self, value: int):
        self._cnt = value

    @abc.abstractmethod
    def find(self, query: PoolQuery) -> Any:
        """
        Find an item by id or name.
        """

    @abc.abstractmethod
    def sample(
        self, size: int, *, query: Optional[PoolQuery] = None
    ) -> Optional[Union[pd.DataFrame, list[ItemT]]]:
        """
        Sample a size of records from the pool.
        size: desired size of the samples
        rule: an optional dictionary specifying a rule or
        a pipeline in mongodb
        query = {
            vehicle_id: str = "VB7",
            driver_id: str = "longfei-zheng",
            dt_start: datetime = None,
            dt_end: datetime = None,
            }
        """

    @abc.abstractmethod
    def __iter__(self) -> Any:
        """
        Iterate over the pool.
        """

    def __getitem__(self, query: PoolQuery) -> Any:
        """
        Get an item by id or name.
        """
        return self.find(query)

    def __len__(self):
        return self.count()

    def __repr__(self):
        return f"Pool(length: {self.count()})"